### Imports

In [1]:
# LLM
from langchain_community.llms import Ollama

# Document loader
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embedding
from langchain_community.embeddings.ollama import OllamaEmbeddings

# Vector store
from langchain.vectorstores import SKLearnVectorStore

# Prompt
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

DATA_PATH = 'data'

In [2]:
model = Ollama(model = 'llama3.1')
embedding = OllamaEmbeddings(model = 'nomic-embed-text')

### Reading and chunking the document

In [3]:
loader = PyPDFDirectoryLoader(DATA_PATH)
documents = loader.load()
        
text_spitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 250,
    chunk_overlap = 0
)

chunks = text_spitter.split_documents(documents)

### Vector Store

In [4]:
vector_store = SKLearnVectorStore.from_documents(
    documents = chunks,    
    embedding = embedding
)

### Using Llama with RAG

In [5]:
prompt_template = """
    Given the following excerpts from a long document and a question, generate a final answer with direct references from the documents ("SOURCES").
    If you don't know the answer, simply say you don't know. Do not attempt to fabricate an answer.

    Use at most three sentences and keep the answer concise and to the point.
    If there are multiple possible answers, select the most relevant one.

    Question: {question}
    =========
    Documents: {documents}
    =========
    Answer:
"""

PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["question", "documents"]
)

chain = RetrievalQA.from_chain_type(
    llm = model,
    retriever = vector_store.as_retriever(k = 4),
    chain_type_kwargs = {
        'prompt': PROMPT,
        'document_variable_name': 'documents'
    }
)

In [8]:
QUERY = 'What is the effect of the Happily Ever After Potion?'
response = chain.invoke(QUERY)
anwser = response['result']

In [9]:
print(f'Question: {QUERY}')
print('\n')
print(f'Anwser: {anwser}')

Question: What is the effect of the Happily Ever After Potion?


Anwser: The effect of the Happily Ever After Potion is to bring happiness, comfort, and beauty to the one who drinks it, including Fiona. According to the potion label, "If one of you drinks this, you both will be fine." (SOURCE: EXT. FOREST13) However, Donkey's concern that something feels off suggests the potion might have unintended consequences.
